In [8]:
import functools

def repeat(num_times):
    def decorator_repeat(func):
        @functools.wraps(func)
        def wrapper_repeat(*args, **kwargs):
            for _ in range(num_times):
                value = func(*args, **kwargs)
            return value
        return wrapper_repeat
    return decorator_repeat

In [9]:
@repeat(num_times=4)
def greet(name):
    print(f"Hello {name}")

In [11]:
greet("Hoss")

Hello Hoss
Hello Hoss
Hello Hoss
Hello Hoss


In [12]:
def repeated(_func=None, *, num_times=2):
    def decorator_repeat(func):
        @functools.wraps(func)
        def wrapper_repeat(*args, **kwargs):
            for _ in range(num_times):
                value = func(*args, **kwargs)
            return value
        return wrapper_repeat

    if _func is None:
        return decorator_repeat
    else:
        return decorator_repeat(_func)

In [13]:
@repeated
def greet(name):
    print(f"Hello {name}")

In [14]:
greet("Hoos")

Hello Hoos
Hello Hoos


In [18]:
@repeated(num_times=3)
def greeter(name):
    print(f"hello there {name}")

In [19]:
greeter("Hoss")

hello there Hoss
hello there Hoss
hello there Hoss


In [20]:
from functools import wraps, partial
import logging

def logged(func=None, *, level=logging.DEBUG, name=None, message=None):
    if func is None:
        return partial(logged, level=level, name=name, message=message)

    logname = name if name else func.__module__
    log = logging.getLogger(logname)
    logmsg = message if message else func.__name__
    @wraps(func)
    def wrapper(*args, **kwargs):
        log.log(level, logmsg)
        return func(*args, **kwargs)
    return wrapper

# Example use
@logged
def add(x, y):
    return x + y

@logged()
def sub(x, y):
    return x - y

@logged(level=logging.CRITICAL, name='example')
def spam():
    print('Spam!')

In [21]:
    logging.basicConfig(level=logging.DEBUG)
    add(2,3)
    sub(2,3)
    spam()

DEBUG:__main__:add
DEBUG:__main__:sub
CRITICAL:example:spam


Spam!


ToDo: try this with the greeter func

In [22]:
import functools

def count_calls(func):
    @functools.wraps(func)
    def wrapper_count_calls(*args, **kwargs):
        wrapper_count_calls.num_calls += 1
        print(f"Call {wrapper_count_calls.num_calls} of {func.__name__!r}")
        return func(*args, **kwargs)
    wrapper_count_calls.num_calls = 0
    return wrapper_count_calls

@count_calls
def say_whee():
    print("Whee!")


say_whee()
say_whee()
say_whee.num_calls

Call 1 of 'say_whee'
Whee!
Call 2 of 'say_whee'
Whee!


2

In [26]:
import functools

class CountCalls:
    def __init__(self, func):
        functools.update_wrapper(self, func)
        self.func = func
        self.num_calls = 0

    def __call__(self, *args, **kwargs):
        self.num_calls += 1
        print(f"Call {self.num_calls} of {self.func.__name__!r}")
        return self.func(*args, **kwargs)

@CountCalls
def say_whee():
    print("Whee!")

@CountCalls
def say_whoo():
    print("Whoo!")

say_whee()
say_whee()

say_whoo()

say_whee()
say_whoo()

Call 1 of 'say_whee'
Whee!
Call 2 of 'say_whee'
Whee!
Call 1 of 'say_whoo'
Whoo!
Call 3 of 'say_whee'
Whee!
Call 2 of 'say_whoo'
Whoo!


In [27]:
import functools
import time

def slow_down(_func=None, *, rate=1):
    """Sleep given amount of seconds before calling the function"""
    def decorator_slow_down(func):
        @functools.wraps(func)
        def wrapper_slow_down(*args, **kwargs):
            time.sleep(rate)
            return func(*args, **kwargs)
        return wrapper_slow_down

    if _func is None:
        return decorator_slow_down
    else:
        return decorator_slow_down(_func)

@slow_down(rate=2)
def countdown(from_number):
    if from_number < 1:
        print("Liftoff!")
    else:
        print(from_number)
        countdown(from_number - 1)


countdown(3)

3
2
1
Liftoff!


In [29]:
import functools

def cache(func):
    """Keep a cache of previous function calls"""
    @functools.wraps(func)
    def wrapper_cache(*args, **kwargs):
        cache_key = args + tuple(kwargs.items())
        if cache_key not in wrapper_cache.cache:
            wrapper_cache.cache[cache_key] = func(*args, **kwargs)
        return wrapper_cache.cache[cache_key]
    wrapper_cache.cache = dict()
    return wrapper_cache

@cache
def fibonacci(num):
    if num < 2:
        return num
    return fibonacci(num - 1) + fibonacci(num - 2)

fibonacci(8)

21

In [30]:
import functools

@functools.lru_cache(maxsize=4)
def fibonacci2(num):
    print(f"Calculating fibonacci({num})")
    if num < 2:
        return num
    return fibonacci2(num - 1) + fibonacci2(num - 2)


fibonacci2(8)

Calculating fibonacci(8)
Calculating fibonacci(7)
Calculating fibonacci(6)
Calculating fibonacci(5)
Calculating fibonacci(4)
Calculating fibonacci(3)
Calculating fibonacci(2)
Calculating fibonacci(1)
Calculating fibonacci(0)


21

In [31]:
fibonacci2.cache_info()

CacheInfo(hits=6, misses=9, maxsize=4, currsize=4)

In [ ]:
def set_unit(unit):
    """Register a unit on a function"""
    def decorator_set_unit(func):
        func.unit = unit
        return func
    return decorator_set_unit

@set_unit("cm^3")
def volume(radius, height):
    return  radius**2 * height

Units become even more powerful and fun when connected with a library that can convert between units. One such library is pint.

In [ ]:
import pint

ureg = pint.UnitRegistry()
vol = volume(3, 5) * ureg(volume.unit)

vol

vol.to("cubic inches")

vol.to("gallons").m  

In [ ]:

def use_unit(unit):
    """Have a function return a Quantity with given unit"""
    use_unit.ureg = pint.UnitRegistry()
    def decorator_use_unit(func):
        @functools.wraps(func)
        def wrapper_use_unit(*args, **kwargs):
            value = func(*args, **kwargs)
            return value * use_unit.ureg(unit)
        return wrapper_use_unit
    return decorator_use_unit

@use_unit("meters per second")
def average_speed(distance, duration):
    return distance / duration

In [ ]:
from flask import Flask, request, abort
import functools
app = Flask(__name__)

def validate_json(*expected_args):                  # 1
    def decorator_validate_json(func):
        @functools.wraps(func)
        def wrapper_validate_json(*args, **kwargs):
            json_object = request.get_json()
            for expected_arg in expected_args:      # 2
                if expected_arg not in json_object:
                    abort(400)
            return func(*args, **kwargs)
        return wrapper_validate_json
    return decorator_validate_json


@app.route("/grade", methods=["POST"])
@validate_json("student_id")
def update_grade():
    json_data = request.get_json()
    # if "student_id" not in json_data:
    #     abort(400)
    # Update database
    return "success!"